# PhD and Supervisor data 

## Clean and restructure dataset

In [ ]:
# Import dependencies
import pandas as pd

# custom functions
from src.clean_names_helpers import (
    remove_non_person_contributors_and_export,
    format_name_to_lastname_firstname, 
    ensure_and_load_spacy_model, 
    merge_near_duplicates_on_col
)

Notebook settings

In [ ]:
# Automatically reloads any modules that are imported, 
# so that any changes made to the module files are reflected # without needing to restart the Jupyter kernel.
# load autoreload module
%load_ext autoreload
# mode 1 reloads only when an import statement is called. For production
# mode 2 reloads before execution of every cell
%autoreload 2

# limit the number of rows that are shown with printing data frames
pd.set_option('display.max_rows', 5)

In [ ]:
# Load, and if not available, download the spacy nlp model 
model_name = "xx_ent_wiki_sm" # multilingual NER model
nlp = ensure_and_load_spacy_model(model_name)

In [ ]:
# Initialize values
NROWS = None # None for all
YEAR_RANGE = range(2011,2022+1) # Years of interest are 2011 to 2022

# names that spacy does not recognize as such
# NOTE: Add the verbatim names here, not the standardized target notation 
# This list can be fed from removed_contributors.csv that is created when running the script
WHITELIST = [ 
    "Oosterlaan, J.",
    "Nollet, F."
    ] 

# non-people's names that don't get filtered out by spaCy 
BLACKLIST = [
    "Cardiology"
]

removed_contributors = []

In [ ]:
# Read data
pairs_raw = pd.read_csv("data/raw/pairs_sups_phds.csv", nrows=NROWS)
pairs_raw = pairs_raw.convert_dtypes() # make sure all integer columns are integer dtype 
pairs_raw

In [ ]:
# remove duplicates
pairs_pruned = pairs_raw.drop_duplicates() 

# Remove rows where 'contributor' is NA
pairs_pruned = pairs_pruned.dropna(subset=['contributor'])

# Filter to the years of interest
pairs_pruned = pairs_pruned.query(
    "year in @YEAR_RANGE"
)

# merge duplicates that only differ on affiliation column
#pairs_pruned = merge_near_duplicates_on_col(pairs_pruned, agg_col='institution')

In [ ]:
# remove contributors that aren't people
csv_path = "data/removed_contributors.csv"
pairs_filtered = remove_non_person_contributors_and_export(pairs_pruned, csv_path, nlp, WHITELIST, BLACKLIST)

print(f"{len(pairs_filtered)} columns are left.")

pairs_filtered

In [ ]:
# Standardize names
pairs_std = pairs_filtered.copy()
# Apply name standardization to the contributor column
pairs_std['contributor'] = pairs_filtered['contributor'].apply(format_name_to_lastname_firstname)

In [ ]:
# Group by publication
aggregated = pairs_std.groupby([
        'integer_id', 
        'thesis_identifier', 
        'institution', 
        'author_name', 
        'title', 
        'year', 
        'language'
    ])
        
# Aggregate contributors into a list
aggregated = aggregated.agg(list)

aggregated = aggregated.reset_index()
    
# make sure the contributor is a sequence from 1 to n_contributors
aggregated['contributor_order'] = aggregated['contributor_order'].apply(lambda lst: list(range(1, len(lst) + 1)))


In [ ]:
# Pivot the dataset, to get to one row per dissertation, with the contributors in columns

# Initialize a list to hold publication data dictionaries
pubs_list = []

# Iterate over each aggregated group
for _, row in aggregated.iterrows():
    # Initialize a dictionary with publication information
    pub_dict = {col: row[col] for col in ['integer_id', 'thesis_identifier', 'institution', 'author_name', 'title', 'year', 'language']}
    
    # Get the list of contributors and their orders for this publication
    contributors = row['contributor']
    contributor_orders = row['contributor_order']
    
    # Add contributors to the dictionary using dynamic keys
    for order in sorted(set(contributor_orders)):  # Ensure unique and sorted order numbers
        if order - 1 < len(contributors):  # Check to prevent index error
            pub_dict[f'contributor_{order}'] = contributors[order - 1]
    
    # Append the publication dictionary to the list
    pubs_list.append(pub_dict)

# Convert the list of dictionaries to a DataFrame
pubs = pd.DataFrame(pubs_list).reset_index(drop=True)

# Ensure correct data types and fill missing values with a suitable placeholder if necessary
pubs = pubs.convert_dtypes()

# Make naming hopefully a bit clearer
pubs.rename(columns={"author_name": "phd_name"}, inplace=True)

pubs

In [ ]:
# Export the dataset for determining the manual gold standard
n_rows_gold_standard = 15
seed = 42 # fixed seed

sampled_pubs = pubs.sample(n=n_rows_gold_standard, random_state=seed)

print(
    f"Exporting {n_rows_gold_standard} rows of {len(pubs)}.\n"
    f"That is {n_rows_gold_standard/len(pubs)*100} percent of the full dataset."
    )

sampled_pubs.to_csv('data/raw/sampled_pubs_for_gold_standard.csv', index=False)

## Diagnose - Check for function duplicate of PhD candidates

In some cases we also get:

- The same PhD listed twice under two different affiliations.
- PhDs that are listed more than 2 times.
- Different versions of the PhDs name (e.g. Podliesna, Svitlana VS Podliesna, S.S.).
- Different versions of the same thesis title.

C.f. [#46](https://github.com/StefKirsch/clean-and-enrich-phd-supervisor-data/issues/46).

In [ ]:
duplicates_title = pubs[pubs.duplicated(subset=["title"], keep=False)].sort_values(by="title")
duplicates_title

In [ ]:
duplicates_name = pubs[pubs.duplicated(subset=["phd_name"], keep=False)].sort_values(by='phd_name')
duplicates_name

## Export Data

In [ ]:
pubs.to_csv('data/cleaned/pubs.csv', index=False)